# Web Scraping con Python
Vamos a usar las librerías [Requests](https://requests.kennethreitz.org/en/master/) y [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) para hacer web scraping. Instalación (en Anaconda):
```
conda install requests
conda install beautifulsoup4
```

In [109]:
import requests
from bs4 import BeautifulSoup

In [110]:
r = requests.get("https://idal.uv.es/ejemplo.html")

In [111]:
type(r)

requests.models.Response

El objeto devuelto es un objeto `requests.models.Response` con unos atributos determinados:

In [112]:
[s for s in dir(r) if not s.startswith('_')]

['apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

Podemos ver el código de respuesta del servidor (útil para detectar errores 4XX o 5XX):

In [113]:
r.status_code

200

La URL de donde ha sacado los datos es:

In [114]:
r.url

'https://idal.uv.es/ejemplo.html'

Para obtener la respuesta completa (código HTML de la página) como una cadena de texto accedemos al atributo `text`:

In [115]:
print(r.text)

<!DOCTYPE html>
<html lang="es">
<head>
  <title>Página de ejemplo</title>
  <meta charset="UTF-8"/>
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/css/bootstrap.min.css">
</head>
<body>

  <div class="jumbotron text-center">
    <h1>Página de ejemplo</h1>
    <p>Página <em>HTML</em> con texto de ejemplo para hacer web scraping</p> 
  </div>
  
  <div class="container">
    <div class="row">
      <div class="col-sm-4">
        <h3>Columna 1</h3>
        <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
        <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
        <a class="btn" href="#" title="IDAL">Enlace 1</a>
      </div>
      <div class="col-sm-4">
        <h3>Columna 2</h3>
        <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
        <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>

Para comprobar el tipo de respuesta (Content Type) y ver si es HTML, JSON, XML, etc.:

In [116]:
r.headers["content-type"]

'text/html; charset=UTF-8'

Si el servidor diera la información de la codificación incorrecta (p. ej. ISO-8859-1 para una página UTF-8) podemos forzar una nueva codificación

In [117]:
r.encoding

'UTF-8'

In [118]:
r.encoding='ISO-8859-1' #cambiamos a ISO-8859-1 para comprobar que muestra mal los acentos
print(r.text)

<!DOCTYPE html>
<html lang="es">
<head>
  <title>PÃ¡gina de ejemplo</title>
  <meta charset="UTF-8"/>
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/css/bootstrap.min.css">
</head>
<body>

  <div class="jumbotron text-center">
    <h1>PÃ¡gina de ejemplo</h1>
    <p>PÃ¡gina <em>HTML</em> con texto de ejemplo para hacer web scraping</p> 
  </div>
  
  <div class="container">
    <div class="row">
      <div class="col-sm-4">
        <h3>Columna 1</h3>
        <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
        <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
        <a class="btn" href="#" title="IDAL">Enlace 1</a>
      </div>
      <div class="col-sm-4">
        <h3>Columna 2</h3>
        <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
        <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...<

Volvemos a la codificación correcta

In [119]:
r.encoding='UTF-8'

Podemos ver todas las cabeceras devueltas:

In [120]:
r.headers

{'Date': 'Thu, 09 Feb 2023 11:04:34 GMT', 'Server': 'Apache/2.4.41 (Ubuntu)', 'Last-Modified': 'Wed, 10 Feb 2021 10:11:14 GMT', 'ETag': '"73d-5baf89f8467b1-gzip"', 'Accept-Ranges': 'bytes', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'Content-Length': '673', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Content-Type': 'text/html; charset=UTF-8'}

O una cabecera en concreto

In [121]:
r.headers['Content-Type']

'text/html; charset=UTF-8'

### Extraer contenido del HTML
Usar expresiones regulares para buscar patrones HTML no está recomendado. Es mejor usar una librería para "parsear" el HTML y extraer el contenido de las etiquetas específicas.  
Aún así, las RegEx se pueden usar para buscar patrones específicos como listas de precios, direcciones de correo, números de teléfono.  
Podemos ejecutar una RegEx en el texto de respuesta para buscar un patrón específico, como p. ej. una URL:

In [122]:
import re

re.findall(r'https?://[\w_./]+', r.text)

['https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/css/bootstrap.min.css',
 'https://idal.uv.es',
 'http://etse.uv.es',
 'http://uv.es']

In [123]:
patron = re.compile(r'https?://[\w_./]+')

re.search(patron, r.text)

<re.Match object; span=(203, 272), match='https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/c>

## Librería BeautifulSoup
La librería BeautifulSoup se utiliza para extraer contenidos desde una página HTML. Tiene una API muy simple a la vez que potente.  
Para empezar, hay que convertir el HTML del texto de respuesta en una estructura DOM jerárquica que se pueda recorrer y buscar.


In [124]:
soup = BeautifulSoup(r.text, "html.parser")

El objeto `soup` contiene el DOM de todo el documento HTML

In [125]:
type(soup)

bs4.BeautifulSoup

Podemos referirnos a cada elemento del DOM por su nombre (etiqueta HTML) dentro del objeto `soup` (si hay varias apariciones sólo devuelve la primera)

In [126]:
soup.title

<title>Página de ejemplo</title>

In [127]:
type(soup.title)

bs4.element.Tag

In [128]:
titulo = soup.title.text
print(titulo)

Página de ejemplo


In [129]:
soup.a

<a class="btn" href="#" title="IDAL">Enlace 1</a>

Atributos del elemento HTML

In [130]:
soup.a['href']

'#'

Si un elemento tiene otros elementos anidados los muestra todos.

In [131]:
soup.div

<div class="jumbotron text-center">
<h1>Página de ejemplo</h1>
<p>Página <em>HTML</em> con texto de ejemplo para hacer web scraping</p>
</div>

El atributo `text` contiene el texto de todos los elementos internos.

In [132]:
soup.div.text

'\nPágina de ejemplo\nPágina HTML con texto de ejemplo para hacer web scraping\n'

In [133]:
soup.div.h1.text

'Página de ejemplo'

El método `find` busca la primera aparición de una etiqueta en el objeto `Soup`. Cada etiqueta contiene todo su contenido dentro de la estructura DOM.

In [134]:
link = soup.find("a")
link

<a class="btn" href="#" title="IDAL">Enlace 1</a>

Es equivalente a:

In [135]:
soup.a

<a class="btn" href="#" title="IDAL">Enlace 1</a>

In [136]:
link.text

'Enlace 1'

De cada elemento podemos obtener sus atributos

In [137]:
link.get('class') #equivalente a link['class']

['btn']

In [138]:
link.attrs

{'class': ['btn'], 'href': '#', 'title': 'IDAL'}

In [139]:
link.attrs['title']

'IDAL'

In [140]:
link['href']

'#'

Podemos encadenar varias etiquetas a buscar

In [141]:
soup.find('p').find('em')

<em>HTML</em>

In [142]:
soup.p.em

<em>HTML</em>

Podemos acotar la búsqueda a una clase específica de la etiqueta (p.ej. `<a class="enlace">...</a>`)

In [143]:
tag = soup.find("a", class_="enlace")
print(tag)

<a class="enlace" href="https://idal.uv.es" title="IDAL">Intelligent Data Analysis Laboratory</a>


In [144]:
#o simplemente
tag = soup.find("a", "enlace")
print(tag)

<a class="enlace" href="https://idal.uv.es" title="IDAL">Intelligent Data Analysis Laboratory</a>


Podemos buscar una etiqueta con un atributo ID específico (p.ej.: `<div id="col4">...</div>`)

In [145]:
tag = soup.find("div", id="col4")
print(tag)

<div class="col-sm-4" id="col4">
<h3>Columna 3</h3>
<p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
<p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
<a class="btn" href="#" title="IDAL">Enlace 3</a>
</div>


Podemos buscar un atributo sin especificar el tipo de elemento que lo contiene:

In [146]:
soup.find(id="enlaces")

<div class="row" id="enlaces">
<h2>Algunos enlaces de prueba</h2>
<ul>
<li><a class="enlace" href="https://idal.uv.es" title="IDAL">Intelligent Data Analysis Laboratory</a></li>
<li><a class="enlace" href="http://etse.uv.es" title="ETSE">Escola Tècnica Superior d'Enginyeria</a></li>
<li><a class="enlace" href="http://uv.es" title="UV">Universitat de València</a></li>
</ul>
</div>

In [147]:
soup.find(href="#")

<a class="btn" href="#" title="IDAL">Enlace 1</a>

In [148]:
soup.find(class_="col-sm-4").h3.text

'Columna 1'

In [149]:
soup.find(class_="col-sm-4")

<div class="col-sm-4">
<h3>Columna 1</h3>
<p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
<p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
<a class="btn" href="#" title="IDAL">Enlace 1</a>
</div>

In [150]:
print(soup.find(class_="col-sm-4").text)


Columna 1
Lorem ipsum dolor sit amet, consectetur adipisicing elit...
Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...
Enlace 1



EL objeto que devuelve `find()` es del tipo `Tag`


In [151]:
type(tag)

bs4.element.Tag

In [152]:
#atributos y métodos de Tag
[s for s in dir(tag) if not s.startswith('_')]

['DEFAULT_INTERESTING_STRING_TYPES',
 'append',
 'attrs',
 'can_be_empty_element',
 'cdata_list_attributes',
 'childGenerator',
 'children',
 'clear',
 'contents',
 'decode',
 'decode_contents',
 'decompose',
 'decomposed',
 'default',
 'descendants',
 'encode',
 'encode_contents',
 'extend',
 'extract',
 'fetchNextSiblings',
 'fetchParents',
 'fetchPrevious',
 'fetchPreviousSiblings',
 'find',
 'findAll',
 'findAllNext',
 'findAllPrevious',
 'findChild',
 'findChildren',
 'findNext',
 'findNextSibling',
 'findNextSiblings',
 'findParent',
 'findParents',
 'findPrevious',
 'findPreviousSibling',
 'findPreviousSiblings',
 'find_all',
 'find_all_next',
 'find_all_previous',
 'find_next',
 'find_next_sibling',
 'find_next_siblings',
 'find_parent',
 'find_parents',
 'find_previous',
 'find_previous_sibling',
 'find_previous_siblings',
 'format_string',
 'formatter_for_name',
 'get',
 'getText',
 'get_attribute_list',
 'get_text',
 'has_attr',
 'has_key',
 'hidden',
 'index',
 'insert',
 '

El método `find_all` devuelve una lista de objetos `Tag` con todas las etiquetas que encuentra con ese parámetro de búsqueda.  
P. ej. para buscar todas las etiquetas de un tipo en el árbol DOM (por ejemplo los enlaces `<a>` de una página):

In [153]:
soup.find_all("a")

[<a class="btn" href="#" title="IDAL">Enlace 1</a>,
 <a class="btn" href="#" title="IDAL">Enlace 2</a>,
 <a class="btn" href="#" title="IDAL">Enlace 3</a>,
 <a class="enlace" href="https://idal.uv.es" title="IDAL">Intelligent Data Analysis Laboratory</a>,
 <a class="enlace" href="http://etse.uv.es" title="ETSE">Escola Tècnica Superior d'Enginyeria</a>,
 <a class="enlace" href="http://uv.es" title="UV">Universitat de València</a>]

Devuelve un elemento especial que funciona como un *iterable*

In [154]:
type(soup.find_all("a"))

bs4.element.ResultSet

In [155]:
type(soup.find_all("a")[0])

bs4.element.Tag

In [156]:
#hay un shortcut para find_all
soup("a")

[<a class="btn" href="#" title="IDAL">Enlace 1</a>,
 <a class="btn" href="#" title="IDAL">Enlace 2</a>,
 <a class="btn" href="#" title="IDAL">Enlace 3</a>,
 <a class="enlace" href="https://idal.uv.es" title="IDAL">Intelligent Data Analysis Laboratory</a>,
 <a class="enlace" href="http://etse.uv.es" title="ETSE">Escola Tècnica Superior d'Enginyeria</a>,
 <a class="enlace" href="http://uv.es" title="UV">Universitat de València</a>]

In [157]:
#No confundir con
soup.a

<a class="btn" href="#" title="IDAL">Enlace 1</a>

Que equivale a

In [158]:
soup("a")[0]

<a class="btn" href="#" title="IDAL">Enlace 1</a>

Podemos especificar una lista de etiquetas a buscar y las devuelve todas en el orden en que aparecen en la página.

In [159]:
soup.find_all(["a", "p"])

[<p>Página <em>HTML</em> con texto de ejemplo para hacer web scraping</p>,
 <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>,
 <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>,
 <a class="btn" href="#" title="IDAL">Enlace 1</a>,
 <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>,
 <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>,
 <a class="btn" href="#" title="IDAL">Enlace 2</a>,
 <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>,
 <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>,
 <a class="btn" href="#" title="IDAL">Enlace 3</a>,
 <a class="enlace" href="https://idal.uv.es" title="IDAL">Intelligent Data Analysis Laboratory</a>,
 <a class="enlace" href="http://etse.uv.es" title="ETSE">Escola Tècnica Superior d'Enginyeria</a>,
 <a class="enlace" href="http://uv.es" title="UV">Universitat de València</a>]

También podemos especificar el criterio de búsqueda de las etiquetas mediante expresiones regulares.

In [160]:
soup.find_all(re.compile("h\d"))

[<h1>Página de ejemplo</h1>,
 <h3>Columna 1</h3>,
 <h3>Columna 2</h3>,
 <h3>Columna 3</h3>,
 <h2>Algunos enlaces de prueba</h2>]

Podemos filtrar por un atributo del `Tag` (por defecto es la clase si no indicamos otra cosa)

In [161]:
for link in soup.find_all('a', class_="enlace"):
    print(link)

<a class="enlace" href="https://idal.uv.es" title="IDAL">Intelligent Data Analysis Laboratory</a>
<a class="enlace" href="http://etse.uv.es" title="ETSE">Escola Tècnica Superior d'Enginyeria</a>
<a class="enlace" href="http://uv.es" title="UV">Universitat de València</a>


Cada elemento de la lista devuelta es del tipo `Tag` con sus atributos particulares.

In [162]:
for link in soup.find_all('a', class_="enlace"):
    print(link["href"])

https://idal.uv.es
http://etse.uv.es
http://uv.es


In [163]:
enlaces = soup.find_all("a", "btn") #equivale a soup.find_all("a", class_="btn")
for e in enlaces:
    print(e)

<a class="btn" href="#" title="IDAL">Enlace 1</a>
<a class="btn" href="#" title="IDAL">Enlace 2</a>
<a class="btn" href="#" title="IDAL">Enlace 3</a>


Podemos hacer una búsqueda de otra etiqueta anidada dentro de la primera búsqueda (útil por ejemplo para buscar elementos genéricos dentro de una sección específica de la página)

In [164]:
tags = soup.find("div", id="col4").find_all("a")
print(tags)

[<a class="btn" href="#" title="IDAL">Enlace 3</a>]


Repetimos esta búsqueda especificando un selector CSS utilizando el método `select` (más simple que lo anterior si se conoce la sintaxis CSS). Este método devuelve una lista con todos los elementos encontrados.    

In [165]:
#dado que
soup.select("#col4") #equivale a soup.find_all(id="col4")

[<div class="col-sm-4" id="col4">
 <h3>Columna 3</h3>
 <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
 <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
 <a class="btn" href="#" title="IDAL">Enlace 3</a>
 </div>]

In [166]:
soup.select("a") #equivale a soup.find_all("a")

[<a class="btn" href="#" title="IDAL">Enlace 1</a>,
 <a class="btn" href="#" title="IDAL">Enlace 2</a>,
 <a class="btn" href="#" title="IDAL">Enlace 3</a>,
 <a class="enlace" href="https://idal.uv.es" title="IDAL">Intelligent Data Analysis Laboratory</a>,
 <a class="enlace" href="http://etse.uv.es" title="ETSE">Escola Tècnica Superior d'Enginyeria</a>,
 <a class="enlace" href="http://uv.es" title="UV">Universitat de València</a>]

P. ej. elementos `p` contenidos dentro de un elemento con ID="col4"

In [167]:
soup.select("#col4 p")

[<p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>,
 <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>]

El atributo `contents` del resultado de una etiqueta individual (objeto `bs4.element.Tag`) contiene una lista de objetos con su contenido interno (que incluye tanto los nodos de texto como la representación e texto de todo el HTML anidado).  
Si el objeto sólo contiene texto es del tipo `NavigableString`, y si contiene etiqueta es del tipo `Tag`. Los saltos de línea (`\n`) entre las etiquetas se consideran elementos de texto y aparecen en la lista devuelta.

In [168]:
print(soup.find("title"))

<title>Página de ejemplo</title>


In [169]:
inner_contents = soup.find("title").contents
print(inner_contents)

['Página de ejemplo']


In [170]:
type(inner_contents)

list

In [171]:
soup.find("div", id="col4")

<div class="col-sm-4" id="col4">
<h3>Columna 3</h3>
<p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
<p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
<a class="btn" href="#" title="IDAL">Enlace 3</a>
</div>

In [172]:
inner_contents = soup.find("div", id="col4").contents
print(inner_contents)

['\n', <h3>Columna 3</h3>, '\n', <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>, '\n', <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>, '\n', <a class="btn" href="#" title="IDAL">Enlace 3</a>, '\n']


In [173]:
inner_contents[0]

'\n'

In [174]:
type(inner_contents[0])

bs4.element.NavigableString

In [175]:
inner_contents[1]

<h3>Columna 3</h3>

In [176]:
type(inner_contents[1])

bs4.element.Tag

El atributo `text` del objeto `bs4.element.Tag` contiene un string con los textos contenidos en la etiqueta, ignorando todas las etiquetas HTML (como por ejemplo `<span>`, `<strong>` o `<i>`):

In [177]:
inner_text = soup.find("div", id="col4").text
inner_text

'\nColumna 3\nLorem ipsum dolor sit amet, consectetur adipisicing elit...\nUt enim ad minim veniam, quis nostrud exercitation ullamco laboris...\nEnlace 3\n'

In [178]:
type(inner_text)

str

In [179]:
print(inner_text.strip())

Columna 3
Lorem ipsum dolor sit amet, consectetur adipisicing elit...
Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...
Enlace 3


Aquí el método `strip()` simplemente elimina las líneas en blanco (`\n`) del resultado.  

### Búsqueda por función
Podemos definir una función booleana para hacer la búsqueda de elementos en el árbol DOM

In [180]:
def etiquetas(tag):
    return tag.has_attr('id')

soup.find_all(etiquetas)

[<div class="col-sm-4" id="col4">
 <h3>Columna 3</h3>
 <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
 <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
 <a class="btn" href="#" title="IDAL">Enlace 3</a>
 </div>,
 <div class="row" id="enlaces">
 <h2>Algunos enlaces de prueba</h2>
 <ul>
 <li><a class="enlace" href="https://idal.uv.es" title="IDAL">Intelligent Data Analysis Laboratory</a></li>
 <li><a class="enlace" href="http://etse.uv.es" title="ETSE">Escola Tècnica Superior d'Enginyeria</a></li>
 <li><a class="enlace" href="http://uv.es" title="UV">Universitat de València</a></li>
 </ul>
 </div>]

In [181]:
[(t.name, t.attrs) for t in soup.find_all(etiquetas)]

[('div', {'class': ['col-sm-4'], 'id': 'col4'}),
 ('div', {'class': ['row'], 'id': 'enlaces'})]

## Navegación por la estructura DOM
Cada elemento `Tag` guarda información de su posición dentro del árbol de etiquetas HTML del documento (estructura DOM) de manera que se puede navegar desde cada etiqueta a sus etiquetas relacionadas:
### Navegar hacia abajo
Podemos usar los nombres de las etiquetas como atributos encadenados del objeto `soup` para recorrer el árbol hacia abajo:

In [182]:
soup.div.h1 #busca la primera etiqueta <h1> dentro de la primera etiqueta <div> 

<h1>Página de ejemplo</h1>

In [183]:
soup("div")[1].h3 #busca la primera etiqueta h3 dentro del elemento 1 de la lista de etiquetas div

<h3>Columna 1</h3>

Todos los hijos de una etiqueta están en su atributo `contents`:

In [184]:
soup.div.contents

['\n',
 <h1>Página de ejemplo</h1>,
 '\n',
 <p>Página <em>HTML</em> con texto de ejemplo para hacer web scraping</p>,
 '\n']

In [185]:
type(soup.div.contents)

list

Los hijos de una etiqueta que sólo contienen texto aparecen en la lista de `contents` como un objeto `NavigableString`. Los que contienen una etiqueta son objetos `Tag`

In [186]:
for t in soup.div.contents:
    print(type(t))

<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>


In [187]:
soup.div.h1.contents

['Página de ejemplo']

In [188]:
type(soup.div.h1.contents[0])

bs4.element.NavigableString

Podemos iterar sobre los descendientes de una etiqueta con el iterador `children`:

In [189]:
soup.div.children

In [190]:
[child for child in soup.div.children]

['\n',
 <h1>Página de ejemplo</h1>,
 '\n',
 <p>Página <em>HTML</em> con texto de ejemplo para hacer web scraping</p>,
 '\n']

In [191]:
#si convertimos el iterador de children a lista es equivalente al método contents
list(soup.div.children)==soup.div.contents

True

Tanto `contents` como `children` consideran sólo los descencientes directos de una etiqueta. Con `descendants` accedemos iterativamente a todos sus descendientes:

In [192]:
soup.div.p

<p>Página <em>HTML</em> con texto de ejemplo para hacer web scraping</p>

In [193]:
soup.div.p.contents

['Página ', <em>HTML</em>, ' con texto de ejemplo para hacer web scraping']

In [194]:
soup.div.p.descendants

<generator object Tag.descendants at 0x000001F27321FC80>

In [195]:
[child for child in soup.div.p.descendants]

['Página ',
 <em>HTML</em>,
 'HTML',
 ' con texto de ejemplo para hacer web scraping']

In [196]:
[child for child in soup.find(id="col4").descendants]

['\n',
 <h3>Columna 3</h3>,
 'Columna 3',
 '\n',
 <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>,
 'Lorem ipsum dolor sit amet, consectetur adipisicing elit...',
 '\n',
 <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>,
 'Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...',
 '\n',
 <a class="btn" href="#" title="IDAL">Enlace 3</a>,
 'Enlace 3',
 '\n']

Para obtener sólo el texto de una etiqueta usamos `string` o para obtener iterativamente todos los textos contenidos usamos el iterador `strings`:

In [197]:
soup.div.h1.string

'Página de ejemplo'

In [198]:
#si una etiqueta contiene otras etiquetas el atributo string está vacío
soup.div.p

<p>Página <em>HTML</em> con texto de ejemplo para hacer web scraping</p>

In [199]:
soup.div.p.string

In [200]:
#iteramos por todos los elementos contenidos para obtener su string
[s for s in soup.div.p.strings]

['Página ', 'HTML', ' con texto de ejemplo para hacer web scraping']

### Navegar hacia arriba
Con `parent` obtenemos la etiqueta superior a una dada, y con `parents` obtenemos iterativamente todos sus ascendientes:

In [201]:
soup.div.h1.parent

<div class="jumbotron text-center">
<h1>Página de ejemplo</h1>
<p>Página <em>HTML</em> con texto de ejemplo para hacer web scraping</p>
</div>

In [202]:
print(soup.find("div", id="col4").parent)

<div class="row">
<div class="col-sm-4">
<h3>Columna 1</h3>
<p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
<p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
<a class="btn" href="#" title="IDAL">Enlace 1</a>
</div>
<div class="col-sm-4">
<h3>Columna 2</h3>
<p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
<p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
<a class="btn" href="#" title="IDAL">Enlace 2</a>
</div>
<div class="col-sm-4" id="col4">
<h3>Columna 3</h3>
<p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
<p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
<a class="btn" href="#" title="IDAL">Enlace 3</a>
</div>
</div>


In [203]:
for p in soup.find("div", id="col4").parents:
    print(p.name, p.attrs)

div {'class': ['row']}
div {'class': ['container']}
body {}
html {'lang': 'es'}
[document] {}


In [204]:
for p in soup.find("h1").parents:
    print(p.name, p.attrs)

div {'class': ['jumbotron', 'text-center']}
body {}
html {'lang': 'es'}
[document] {}


### Navegar hacia los lados
Los métodos `next_sibling` y `previous_sibling` acceden a la etiqueta posterior o anterior del mismo nivel. Los métodos `next_siblings` y `previous_siblings` lo hacen iterativamente con todos los elementos al mismo nivel.

In [205]:
soup.find("div", id="enlaces")

<div class="row" id="enlaces">
<h2>Algunos enlaces de prueba</h2>
<ul>
<li><a class="enlace" href="https://idal.uv.es" title="IDAL">Intelligent Data Analysis Laboratory</a></li>
<li><a class="enlace" href="http://etse.uv.es" title="ETSE">Escola Tècnica Superior d'Enginyeria</a></li>
<li><a class="enlace" href="http://uv.es" title="UV">Universitat de València</a></li>
</ul>
</div>

In [206]:
soup.find("div", id="enlaces").ul.contents

['\n',
 <li><a class="enlace" href="https://idal.uv.es" title="IDAL">Intelligent Data Analysis Laboratory</a></li>,
 '\n',
 <li><a class="enlace" href="http://etse.uv.es" title="ETSE">Escola Tècnica Superior d'Enginyeria</a></li>,
 '\n',
 <li><a class="enlace" href="http://uv.es" title="UV">Universitat de València</a></li>,
 '\n']

In [207]:
soup.find("div", id="enlaces").li

<li><a class="enlace" href="https://idal.uv.es" title="IDAL">Intelligent Data Analysis Laboratory</a></li>

In [208]:
soup.find("div", id="enlaces").li.previous_sibling

'\n'

In [209]:
soup.find("div", id="enlaces").li.next_sibling.next_sibling

<li><a class="enlace" href="http://etse.uv.es" title="ETSE">Escola Tècnica Superior d'Enginyeria</a></li>

In [210]:
#buscamos elementos hermanos a continuación del <li> intermedio
enlace_etse = soup.find("div", id="enlaces").find("a",{"title": "ETSE"})
enlace_etse

<a class="enlace" href="http://etse.uv.es" title="ETSE">Escola Tècnica Superior d'Enginyeria</a>

In [211]:
[tag for tag in enlace_etse.parent.next_siblings]

['\n',
 <li><a class="enlace" href="http://uv.es" title="UV">Universitat de València</a></li>,
 '\n']

In [212]:
#buscamos elementos hermanos anteriores al último <li>

[tag for tag in soup.find("div", id="enlaces").find("a",{"title": "UV"}).parent.previous_siblings]

['\n',
 <li><a class="enlace" href="http://etse.uv.es" title="ETSE">Escola Tècnica Superior d'Enginyeria</a></li>,
 '\n',
 <li><a class="enlace" href="https://idal.uv.es" title="IDAL">Intelligent Data Analysis Laboratory</a></li>,
 '\n']

### Ejemplo completo
Del siguiente código HTML vamos a extraer los ítems listados y su precio

In [213]:
texto = """
<body>
<div id="listings_prices">
 <div class="item">
  <li class="item_name">Watch</li>
  <div class="main_price">Price: $66.68</div>
  <div class="discounted_price">Discounted price: $46.68</div>
   </div>
   <div class="item">
  <li class="item_name">Watch2</li>
  <div class="main_price">Price: $56.68</div>
   </div>
</div>
</body>
"""

In [214]:
soup = BeautifulSoup(texto, "html.parser")
for product in soup.find_all("div", "item"):
    print(product.find("li").text)

Watch
Watch2


In [215]:
soup = BeautifulSoup(texto, "html.parser")
for product in soup.find_all("div", "item"):
    product_title = product.find("li").text 
    product_price = re.search(r'\$\d+\.\d+', product.find("div", "main_price").text)[0]
    print(f"{product_title} is selling for {product_price}")

Watch is selling for $66.68
Watch2 is selling for $56.68
